In [2]:
import os
from optparse import OptionParser
import sys
import numpy as np
import pandas as pd
import json
import yaml

In [3]:
sys.path.insert(0, '/home/jgutman/mvesc/ETL')
sys.path.insert(0, '/home/jgutman/mvesc/Features')
sys.path.insert(0, '/home/jgutman/mvesc/Models_Results')

In [4]:
!git branch

  adding_allTeachers_to_database
  copy_df2postgres_file_to_utility
  error_analysis
  generate_normalized_test_scores
  gpa_generation
  intervention_dataETL_feature_generation
  master
* more_feature_debugging_and_pipeline_testing


In [5]:
from mvesc_utility_functions import *

In [6]:
from sklearn.cross_validation import *

In [7]:
from estimate_prediction_model import *

In [8]:
model_options = read_in_yaml('/home/jgutman/mvesc/Models_Results/model_options_expand_features.yaml')

In [9]:
model_options['feature_grade_range'] = [8,9]
model_options['cohorts_training'] = [2007, 2008, 2009, 2010]

In [10]:
model_options['features_included'] = {
    'demographics': ['ethnicity', 'gender'], 
    'absence': ['absence*', 'absence_unexcused*', 'tardy*', 'tardy_unexcused*', 'medical*', 
                'absence_consec*', 'tardy_consec*'],
    'grades': ['gpa*', 'language_gpa*', 'stem_gpa*', 'humanities_gpa*', 'art_gpa*', 'health_gpa*', 
        'future_prep_gpa*', 'interventions_gpa*', 'language_gpa*', 'num_language_classes*', 'num_stem_classes*',
        'num_humanities_classes*', 'num_art_classes*', 'num_health_classes*', 'num_future_prep_classes*',
        'num_interventions_classes*', 'num_language_classes*', 'percent_passed_pf_classes*', 'num_pf_classes*',
        'gpa_district*'], 
    'mobility': ['n_addresses_to*', 'n_districts_to*', 'n_cities_to*', 'n_records_to*', 'avg_address_change_to*',
        'avg_district_change_to*', 'avg_city_change_to*', 'street_transition_in*', 'district_transition_in*',
        'city_transition_in*', 'mid_year_withdraw*'],
    'snapshots': ['disadvantagement*', 'disability*', 'gifted*', 'iss*', 'oss*', 'limited_english*', 
        'special_ed*', 'discipline_incidents*']
}

In [11]:
outcome_plus_features = build_outcomes_plus_features(model_options)

In [12]:
outcome_plus_features.dropna(subset=[model_options['outcome_name'],
        model_options['cohort_grade_level_begin']], inplace=True)

In [13]:
train, test = temporal_cohort_test_split(outcome_plus_features,
            model_options['cohort_grade_level_begin'],
            model_options['cohorts_held_out'],
            model_options['cohorts_training'])

In [14]:
print(train.shape)
print(test.shape)

(4193, 146)
(2009, 146)


In [15]:
train_X = train.drop([model_options['outcome_name'],
        model_options['cohort_grade_level_begin']],axis=1)
test_X = test.drop([model_options['outcome_name'],
        model_options['cohort_grade_level_begin']],axis=1)
train_y = train[model_options['outcome_name']]
test_y = test[model_options['outcome_name']]

In [16]:
# do missing value feature imputation here
train_X, test_X = impute_missing_values(train_X, test_X,
    model_options['missing_impute_strategy'])
assert (all(train_X.columns == test_X.columns)),\
    "train and test have different columns"

In [17]:
# do feature scaling here
train_X, test_X = scale_features(train_X, test_X,
    model_options['feature_scaling'])
assert (all(train_X.columns == test_X.columns)),\
    "train and test have different columns"

In [49]:
cohort_kfolds = LeaveOneLabelOut(train[
        model_options['cohort_grade_level_begin']])
print(cohort_kfolds)

sklearn.cross_validation.LeaveOneLabelOut(labels=[2007 2007 2007 ..., 2010 2010 2010])


In [19]:
len(cohort_kfolds)

4

In [20]:
print(pd.unique(train.cohort_9th))
print(pd.unique(test.cohort_9th))

[2007 2008 2009 2010]
[2011]


In [61]:
restructured_folds = []
for train_list, test_list in cohort_kfolds:
    test_year = pd.unique(cohort_kfolds.labels[test_list])
    train_years_after_test = cohort_kfolds.labels[train_list] > test_year
    train_indices_after_test = np.where(train_years_after_test)
    train_list = np.delete(train_list, train_indices_after_test)
    fold = (train_list, test_list)
    if len(train_list) > 0:
        restructured_folds.append(fold)

In [52]:
for train_list, test_list in cohort_kfolds:
    print(len(train_list))

3142
3178
3172
3087


In [58]:
type(list(cohort_kfolds)[0][0])

numpy.ndarray

In [48]:
np.delete(train_list, 0)

array([1052, 1053, 1054, ..., 4190, 4191, 4192])

In [62]:
restructured_folds

[(array([   0,    1,    2, ..., 1048, 1049, 1050]),
  array([1051, 1052, 1053, ..., 2063, 2064, 2065])),
 (array([   0,    1,    2, ..., 2063, 2064, 2065]),
  array([2066, 2067, 2068, ..., 3084, 3085, 3086])),
 (array([   0,    1,    2, ..., 3084, 3085, 3086]),
  array([3087, 3088, 3089, ..., 4190, 4191, 4192]))]